[![Binder](https://mybinder.org/badge_logo.svg)](https://lab.mlpack.org/v2/gh/mlpack/examples/master?urlpath=lab%2Ftree%2Freinforcement_learning_gym%2Flunar_lander_dqn%2Flunar_lander_dqn.ipynb)

You can easily run this notebook at https://lab.mlpack.org/

This notebook shows how to get started with training reinforcement learning agents, particularly DQN agents, using mlpack. Here, we train a [Simple DQN](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf) agent to get high scores for the [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/) environment. 

We make the agent train and test on OpenAI Gym toolkit's GUI interface provided through a distributed infrastructure (TCP API). More details can be found [here](https://github.com/zoq/gym_tcp_api).

A video of the trained agent can be seen in the end.

## Including necessary libraries and namespaces

In [1]:
#include <mlpack/xeus-cling.hpp>

In [2]:
#include <mlpack/core.hpp>
#include <mlpack/methods/ann/ffn.hpp>
#include <mlpack/methods/reinforcement_learning/q_learning.hpp>
#include <mlpack/methods/reinforcement_learning/q_networks/simple_dqn.hpp>
#include <mlpack/methods/reinforcement_learning/environment/env_type.hpp>
#include <mlpack/methods/reinforcement_learning/policy/greedy_policy.hpp>
#include <mlpack/methods/reinforcement_learning/training_config.hpp>

In [3]:
// Used to run the agent on gym's environment (provided externally) for testing.
#include <gym/environment.hpp>

In [4]:
// Used to generate and display a video of the trained agent.
#include "xwidgets/ximage.hpp"
#include "xwidgets/xvideo.hpp"
#include "xwidgets/xaudio.hpp"

In [5]:
using namespace mlpack;

In [6]:
using namespace mlpack::ann;

In [7]:
using namespace ens;

In [8]:
using namespace mlpack::rl;

## Initializing the agent

In [9]:
// Set up the state and action space.
DiscreteActionEnv::State::dimension = 8;
DiscreteActionEnv::Action::size = 4;

In [10]:
// Set up the network.
FFN<MeanSquaredError<>, GaussianInitialization> network(
    MeanSquaredError<>(), GaussianInitialization(0, 1));
network.Add<Linear<>>(DiscreteActionEnv::State::dimension, 128);
network.Add<ReLULayer<>>();
network.Add<Linear<>>(128, DiscreteActionEnv::Action::size);

SimpleDQN<> model(network);

In [11]:
// Set up the policy and replay method.
GreedyPolicy<DiscreteActionEnv> policy(1.0, 2000, 0.1, 0.99);
RandomReplay<DiscreteActionEnv> replayMethod(64, 100000);

In [12]:
// Set up training configurations.
TrainingConfig config;
config.ExplorationSteps() = 100;
config.DoubleQLearning() = false;

In [13]:
// Set up DQN agent.
QLearning<DiscreteActionEnv, decltype(model), AdamUpdate, decltype(policy)>
    agent(config, model, policy, replayMethod);

## Preparation for training the agent

In [14]:
// Set up the gym training environment.
gym::Environment env("gym.kurg.org", "4040", "LunarLander-v2");

// Initializing training variables.
std::vector<double> returnList;
size_t episodes = 0;
bool converged = true;

// The number of episode returns to keep track of.
size_t consecutiveEpisodes = 50;

In [15]:
// Function to train the agent on the gym implementation of LunarLander environment.
void Train(const size_t numSteps)
{
  agent.Deterministic() = false;
  std::cout << "Training for " << numSteps << " steps." << std::endl;
  while (agent.TotalSteps() < numSteps)
  {
    double episodeReturn = 0;
    env.reset();
    do
    {
      agent.State().Data() = env.observation;
      agent.SelectAction();
      arma::mat action = {double(agent.Action().action)};

      env.step(action);
      DiscreteActionEnv::State nextState;
      nextState.Data() = env.observation;

      replayMethod.Store(agent.State(), agent.Action(), env.reward, nextState,
          env.done, 0.99);
      episodeReturn += env.reward;
      agent.TotalSteps()++;
      if (agent.Deterministic() || agent.TotalSteps() < config.ExplorationSteps())
        continue;
      agent.TrainAgent();
    } while (!env.done);
    returnList.push_back(episodeReturn);
    episodes += 1;

    if (returnList.size() > consecutiveEpisodes)
      returnList.erase(returnList.begin());
        
    double averageReturn = std::accumulate(returnList.begin(),
                                           returnList.end(), 0.0) /
                           returnList.size();
    if(episodes % 5 == 0)
    {
      std::cout << "Avg return in last " << returnList.size()
          << " episodes: " << averageReturn
          << "\t" << episodes
          << "th episode return: " << episodeReturn
          << "\t Steps: " << agent.TotalSteps() << std::endl;
    }
  }
}

## Let the training begin

In [16]:
// Training the agent for a total of at least 10000 steps.
Train(10000)

Training for 10000 steps.
Avg return in last 5 episodes: -194.18	5th episode return: -153.237	 Steps: 441
Avg return in last 10 episodes: -182.628	10th episode return: -101.164	 Steps: 1028
Avg return in last 15 episodes: -177.656	15th episode return: -200.554	 Steps: 1764
Avg return in last 20 episodes: -165.164	20th episode return: -195.816	 Steps: 2742
Avg return in last 25 episodes: -173.573	25th episode return: -318.202	 Steps: 3718
Avg return in last 30 episodes: -166.942	30th episode return: -79.2901	 Steps: 4654
Avg return in last 35 episodes: -159.46	35th episode return: -75.691	 Steps: 5423
Avg return in last 40 episodes: -151.316	40th episode return: -149.785	 Steps: 6649


## Testing the trained agent

In [17]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "LunarLander-v2");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();
std::cout << url;
auto video = xw::video_from_url(url).finalize();
video

 Total steps: 687	 Total reward: -461.826
https://gym.kurg.org/dd6d73ef7b7f4/output.webm

A Jupyter widget

## A little more training...

In [18]:
// Training the same agent for a total of at least 100000 steps.
Train(100000)

Training for 100000 steps.
Avg return in last 45 episodes: -146.766	45th episode return: -252.637	 Steps: 10917
Avg return in last 50 episodes: -141.891	50th episode return: 18.9724	 Steps: 13376
Avg return in last 50 episodes: -137.465	55th episode return: -238.157	 Steps: 17036
Avg return in last 50 episodes: -125.792	60th episode return: -45.9381	 Steps: 19453
Avg return in last 50 episodes: -123.141	65th episode return: -175.618	 Steps: 20381
Avg return in last 50 episodes: -115.198	70th episode return: -144.099	 Steps: 21934
Avg return in last 50 episodes: -101.778	75th episode return: -12.9555	 Steps: 23576
Avg return in last 50 episodes: -97.059	80th episode return: -61.9758	 Steps: 24151
Avg return in last 50 episodes: -96.595	85th episode return: 75.604	 Steps: 25536
Avg return in last 50 episodes: -104.601	90th episode return: -81.2942	 Steps: 26192
Avg return in last 50 episodes: -105.909	95th episode return: -284.902	 Steps: 26709
Avg return in last 50 episodes: -122.314	10

# Final agent testing!
*Note*: If you don't find a satisfactory output, please rerun the cell below. It's not guaranteed that the agent will receive high rewards on all test runs.

In [19]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "LunarLander-v2");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();
std::cout << url;
auto video = xw::video_from_url(url).finalize();
video

 Total steps: 400	 Total reward: 205.182
https://gym.kurg.org/dd039a77afe24/output.webm

A Jupyter widget